In [4]:
from datasets import load_dataset
from transformers import pipeline
from tqdm import tqdm
import torch
import pandas as pd
from sklearn.metrics import classification_report

In [5]:
dataset = load_dataset('hpe-ai/medical-cases-classification-tutorial')
df = dataset['train'].to_pandas()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/192 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


medical_cases_train.csv:   0%|          | 0.00/6.34M [00:00<?, ?B/s]

medical_cases_validation.csv:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

medical_cases_test.csv:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1724 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/370 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/370 [00:00<?, ? examples/s]

In [6]:
df.head()

,description,transcription,sample_name,medical_specialty,keywords
0,Pacemaker ICD interrogation. Severe nonischem...,"PROCEDURE NOTE: , Pacemaker ICD interrogation....",Pacemaker Interrogation,Cardiovascular / Pulmonary,"cardiovascular / pulmonary, cardiomyopathy, ve..."
1,"Erythema of the right knee and leg, possible s...","PREOPERATIVE DIAGNOSES: , Erythema of the righ...",Aspiration - Knee Joint,Orthopedic,"orthopedic, knee and leg, anterolateral portal..."
2,Left cardiac catheterization with selective ri...,"PREOPERATIVE DIAGNOSIS: , Post infarct angina....",Cardiac Cath & Selective Coronary Angiography,Cardiovascular / Pulmonary,"cardiovascular / pulmonary, selective, angiogr..."
3,Patient with a history of coronary artery dise...,"REASON FOR VISIT: , Acute kidney failure.,HIST...",Acute Kidney Failure,Nephrology,None
4,Cardiac evaluation and treatment in a patient ...,"REASON FOR REFERRAL: , Cardiac evaluation and ...",Cardiac Consultation - 6,Cardiovascular / Pulmonary,None


In [7]:
def simplify_specialty(specialty):
    return specialty.split("/")[0].split("-")[0].strip()

df['medical_specialty'] = df['medical_specialty'].apply(simplify_specialty)

In [ ]:
# def zero_shot_classify(pipe, sample_name, description, transcription, keywords):

#   # categories = '\n'.join(f"- {cat}" for cat in df['medical_specialty'].unique())
#   prompt = f"""
#     You are a medical assistant specializing in categorizing clinical cases into appropriate medical specialties, given information like description, transcription and keywords.

#     Your task is to read the provided case details and identify the correct medical specialty.

#     Respond with only one of the following categories (no extra text or explanation):
#     {df['medical_specialty'].unique()}

#     Only respond with the category name, no punctuation, no quotes.

#     ---

#     Sample Name: {sample_name}

#     Description:
#     {description}

#     Transcription:
#     {transcription}

#     Keywords:
#     {keywords}

#     ---

#     Answer:
#   """


#   output = pipe(prompt)[0]['generated_text']
#   return output

In [8]:
def clean_prediction(prediction):
    prediction = prediction.strip().split("\n")[0]  # Get only the first line
    prediction = prediction.replace("-", " ").replace(",", " ")
    words = prediction.split()

    # Try to find the first matching specialty from your dataset
    for word in words:
        for specialty in df['medical_specialty'].unique():
            if word.lower() in specialty.lower() or specialty.lower() in word.lower():
                return specialty

    # print(prediction)
    print(f'Wrong word: {prediction}')
    return "Unknown"


In [9]:
def classify_medical_specialty(pipe, current_sample, df, all_categories, n_shot=0):

    few_shot_examples = []
    for category in all_categories:
        class_samples = df[df['medical_specialty'] == category]
        sampled = class_samples.sample(n=min(n_shot, len(class_samples)), random_state=42)
        few_shot_examples.extend(sampled.to_dict(orient='records'))

    # print(few_shot_examples)
    prompt = f"""You are a medical assistant. Given the description, your job is to classify each medical case into ONLY ONE of the following specialties:
    {', '.join(all_categories)}

      Respond with ONLY the name of the correct medical specialty from the given list only (no word should be out of list).
    """

    for i, ex in enumerate(few_shot_examples):
        prompt += f"""
          Example {i+1}:
          Description: {ex['description']}
          medical specialty': {ex['medical_specialty']}
        """

    prompt += f"""

    Now, classify the following:

    Description: {current_sample['description']}

    medical specialty:"""

    output = pipe(prompt)[0]["generated_text"]

    # Clean the output to match known categories
    # return clean_prediction(output)
    return clean_prediction((output.split('medical specialty:')[1]))
    # return output
    # return clean_prediction(prediction)


In [10]:
from transformers import AutoTokenizer, pipeline

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

pipe = pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    device_map="cuda",
    max_new_tokens=15,
    pad_token_id=tokenizer.eos_token_id  # Avoid warning
)

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda


In [11]:
def n_shot_classify(n_shot):
  pred = []
  for i in tqdm(range(len(df))):

    row = df.iloc[i]

    output = classify_medical_specialty(pipe, row, df, df['medical_specialty'].unique(), n_shot=n_shot)

    torch.cuda.empty_cache()

    pred.append(output)
  return pred

In [12]:
pred_zero_shot = n_shot_classify(0)

  0%|          | 1/1724 [00:02<1:07:17,  2.34s/it]

Wrong word: ?


  0%|          | 2/1724 [00:02<38:41,  1.35s/it]  

Wrong word: __________


  0%|          | 3/1724 [00:03<29:37,  1.03s/it]

...

Wrong word: [Your answer]


100%|█████████▉| 1723/1724 [20:31<00:00,  1.78it/s]

Wrong word: ?


100%|██████████| 1724/1724 [20:32<00:00,  1.40it/s]

Wrong word: __________


In [13]:
print(classification_report(pred_zero_shot,df['medical_specialty']))

                  precision    recall  f1-score   support

  Cardiovascular       0.00      0.50      0.01         4
             ENT       0.00      0.00      0.00         0
Gastroenterology       0.00      0.00      0.00         0
      Hematology       0.00      0.00      0.00         2
      Nephrology       0.00      0.00      0.00         0
       Neurology       0.00      0.00      0.00         0
    Neurosurgery       0.00      0.00      0.00         0
      Obstetrics       0.00      0.00      0.00         0
   Ophthalmology       0.00      0.00      0.00         0
      Orthopedic       0.00      0.00      0.00         0
      Pediatrics       0.00      0.00      0.00         0
      Psychiatry       0.02      1.00      0.04         1
       Radiology       0.00      0.00      0.00         0
         Unknown       0.00      0.00      0.00      1717

        accuracy                           0.00      1724
       macro avg       0.00      0.11      0.00      1724
    weighted

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [16]:
pred_five_shot = n_shot_classify(5)

  0%|          | 1/1724 [00:03<1:36:53,  3.37s/it]

Wrong word: ________


  0%|          | 2/1724 [00:06<1:36:08,  3.35s/it]

Wrong word: ?


  0%|          | 3/1724 [00:10<1:36:00,  3.35s/it]

...

Wrong word: __________


100%|█████████▉| 1723/1724 [1:38:22<00:03,  3.33s/it]

Wrong word: _______


100%|██████████| 1724/1724 [1:38:25<00:00,  3.43s/it]

Wrong word: ?


In [17]:
print(classification_report(pred_five_shot,df['medical_specialty']))

                  precision    recall  f1-score   support

  Cardiovascular       0.00      0.00      0.00       0.0
             ENT       0.00      0.00      0.00       0.0
Gastroenterology       0.00      0.00      0.00       0.0
      Hematology       0.00      0.00      0.00       1.0
      Nephrology       0.00      0.00      0.00       0.0
       Neurology       0.00      0.00      0.00       0.0
    Neurosurgery       0.00      0.00      0.00       0.0
      Obstetrics       0.00      0.00      0.00       0.0
   Ophthalmology       0.00      0.00      0.00       0.0
      Orthopedic       0.00      0.00      0.00       0.0
      Pediatrics       0.00      0.00      0.00       0.0
      Psychiatry       0.00      0.00      0.00       0.0
       Radiology       0.00      0.00      0.00       0.0
         Unknown       0.00      0.00      0.00    1723.0

        accuracy                           0.00    1724.0
       macro avg       0.00      0.00      0.00    1724.0
    weighted

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [14]:
pred_three_shot = n_shot_classify(3)

  0%|          | 1/1724 [00:02<1:06:57,  2.33s/it]

Wrong word: ?


  0%|          | 2/1724 [00:04<1:05:57,  2.30s/it]

Wrong word: _______


  0%|          | 3/1724 [00:07<1:08:33,  2.39s/it]

...

Wrong word: [ ]


100%|█████████▉| 1723/1724 [1:06:10<00:02,  2.33s/it]

Wrong word: __________


100%|██████████| 1724/1724 [1:06:12<00:00,  2.30s/it]

Wrong word: __________


In [18]:
print(classification_report(pred_three_shot,df['medical_specialty']))

                  precision    recall  f1-score   support

  Cardiovascular       0.00      0.29      0.01         7
             ENT       0.00      0.00      0.00         0
Gastroenterology       0.00      0.00      0.00         0
      Hematology       0.00      0.00      0.00         2
      Nephrology       0.00      0.00      0.00         0
       Neurology       0.00      0.00      0.00         0
    Neurosurgery       0.00      0.00      0.00         0
      Obstetrics       0.00      0.00      0.00         0
   Ophthalmology       0.00      0.00      0.00         0
      Orthopedic       0.00      0.00      0.00         1
      Pediatrics       0.00      0.00      0.00         0
      Psychiatry       0.00      0.00      0.00         0
       Radiology       0.00      0.00      0.00         0
         Unknown       0.00      0.00      0.00      1714

        accuracy                           0.00      1724
       macro avg       0.00      0.02      0.00      1724
    weighted

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_